# <center>Implementing LDA in Python</center>

<center>Dr. W.J.B. Mattingly</center>

<center>Smithsonian Data Science Lab and United States Holocaust Memorial Museum</center>

<center>February 2021</center>

## Key Concepts in this Notebook

## Introduction

## Importing the Required Libraries

In [1]:
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#1introduction
import numpy as np
import pandas as pd
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
# import pyLDAvis.gensim
import pyLDAvis.gensim_models as gensimvis

from googletrans import Translator
translator = Translator()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Preparing the Data

In [2]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f) 
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)


In [3]:
stopwords = stopwords.words("english")

In [4]:
print (stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [5]:
# data = load_data("data/ushmm_dn.json")["texts"]

url = '/Users/dj/Documents/GitHub/out.csv'
df = pd.read_csv(url)

data = [df.iloc[i,0] for i in np.arange(100)]

print (data[0][0:90])

Onderzoek van de zaakDit arrest is gewezen naar aanleiding van het onderzoek ter terechtzi


In [6]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0][0:90])

arrest gewezen aanleide onderzoek ter terechtzitte hoger voormeld het hof heeft kennisgeno


In [7]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

['arrest', 'gewezen', 'aanleide', 'onderzoek', 'ter', 'terechtzitte', 'hoger', 'voormeld', 'het', 'hof', 'heeft', 'kennisgenoman', 'vordere', 'generaal', 'raadsman', 'gebracht']


In [8]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)]
aanleide


In [9]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")


## Vizualizing the Data

In [10]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word, mds="mmds", R=10)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.016995  0.071953       1        1  52.037085
0      0.068344 -0.082237       2        1  26.372386
1     -0.085339  0.010284       3        1  21.590529, topic_info=             Term        Freq       Total Category  logprob  loglift
13   terechtzitte   53.000000   53.000000  Default  10.0000  10.0000
367   slachtoffer   59.000000   59.000000  Default   9.0000   9.0000
282     verklaart   26.000000   26.000000  Default   8.0000   8.0000
4         gewezen   21.000000   21.000000  Default   7.0000   7.0000
6             het  479.000000  479.000000  Default   6.0000   6.0000
..            ...         ...         ...      ...      ...      ...
7             hof   32.744197  151.959420   Topic3  -4.0072  -0.0020
27             op   23.498069  127.826207   Topic3  -4.3390  -0.1608
56            die   21.901789  111.586919   Topic3  -4.4093  -0.0953
44             al   18.075080   67.043544   Topic3  -4.6014   0.2221
128          voor   18.179162   97.465668   Topic3  -4.5956  -0.1463

[85 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
0         1  0.135706  aanleide
0         2  0.780312  aanleide
0         3  0.101780  aanleide
1380      2  0.923640      akte
44        1  0.492217        al
...     ...       ...       ...
42        1  0.604237    worden
42        2  0.189565    worden
42        3  0.213260    worden
285       1  0.087729     zitte
285       2  0.877285     zitte

[118 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2])